# Prototypes and Inheritance in JavaScript

Разбиране на прототипно-базирания модел на наследяване в JavaScript.

## 1. What are Prototypes?

В JavaScript всеки обект има скрита връзка към друг обект, наречен негов **прототип**. Когато се опитате да получите достъп до свойство на обект, JavaScript първо търси в самия обект. Ако не го намери, търси в прототипа на обекта, след това в прототипа на прототипа и така нататък - образувайки **прототипна верига**.

Мислете за това като за наследяване в родословно дърво: ако нямате нещо, можете да го заемете от родителя си, а ако той го няма, от неговия родител и т.н.

In [ ]:
// Every object has a prototype
const obj = { a: 1 };
console.log(Object.getPrototypeOf(obj));  // Object.prototype

// Arrays have Array.prototype
const arr = [1, 2, 3];
console.log(Object.getPrototypeOf(arr));  // Array.prototype

// Functions have Function.prototype
function fn() {}
console.log(Object.getPrototypeOf(fn));   // Function.prototype

## 2. The Prototype Chain (ELI5)

Представете си, че търсите играчка:
1. Първо, проверявате в собствената си кутия за играчки
2. Ако я няма, питате по-големия си брат/сестра
3. Ако и те я нямат, питат родителите ви
4. Ако родителите я нямат, питат баба и дядо
5. В крайна сметка стигате до края (null)

Точно така работи прототипната верига на JavaScript!

In [ ]:
const grandparent = { surname: "Smith" };
const parent = Object.create(grandparent);
parent.job = "Engineer";
const child = Object.create(parent);
child.name = "Alice";

console.log(child.name);     // "Alice" (own property)
console.log(child.job);      // "Engineer" (from parent)
console.log(child.surname);  // "Smith" (from grandparent)

// The chain: child -> parent -> grandparent -> Object.prototype -> null

## 3. Constructor Functions and Prototypes

Конструкторните функции се използват за създаване на обекти. Чрез добавяне на методи към `prototype` свойството на конструктора, всички инстанции, създадени от този конструктор, могат да споделят едни и същи методи, спестявайки памет.

In [ ]:
function Person(name, age) {
    this.name = name;
    this.age = age;
}

// Add methods to prototype (shared by all instances)
Person.prototype.greet = function() {
    return `Hi, I'm ${this.name}`;
};

const alice = new Person("Alice", 25);
const bob = new Person("Bob", 30);

console.log(alice.greet());  // "Hi, I'm Alice"
console.log(bob.greet());    // "Hi, I'm Bob"

// Both share the same greet method
console.log(alice.greet === bob.greet);  // true

## 4. Understanding `__proto__` vs `prototype`

- `prototype`: Свойство на конструкторните функции, което определя какво ще наследят инстанциите
- `__proto__`: Свойство на обектите, което сочи към действителния прототипен обект

Мислете за това по следния начин:
- `prototype` е планът
- `__proto__` е действителната връзка

In [ ]:
function Dog(name) {
    this.name = name;
}

Dog.prototype.bark = function() {
    return "Woof!";
};

const rex = new Dog("Rex");

// Dog.prototype is the blueprint
console.log(Dog.prototype);

// rex.__proto__ points to Dog.prototype
console.log(rex.__proto__ === Dog.prototype);  // true

// rex can use bark because of the prototype chain
console.log(rex.bark());  // "Woof!"

## 5. Object.create() and Prototype Manipulation

Методът `Object.create()` ви позволява да създадете нов обект и изрично да зададете неговия прототип. Това предоставя директен начин за контрол на прототипната верига и установяване на наследяване между обекти.

In [ ]:
// Object.create() creates object with specified prototype
const animal = {
    eat: function() {
        return "Eating...";
    }
};

const dog = Object.create(animal);
dog.bark = function() {
    return "Woof!";
};

console.log(dog.bark());  // "Woof!" (own method)
console.log(dog.eat());   // "Eating..." (from prototype)

// Check prototype chain
console.log(Object.getPrototypeOf(dog) === animal);  // true

## 6. Inheritance Patterns

Тази секция илюстрира класически модел на наследяване в JavaScript преди ES6 класовете. Той включва извикване на родителския конструктор с `.call()` и задаване на прототипа на детето да бъде обект, създаден от прототипа на родителя, използвайки `Object.create()`, за да се установи прототипната верига.

In [ ]:
// Parent constructor
function Animal(name) {
    this.name = name;
}

Animal.prototype.eat = function() {
    return `${this.name} is eating`;
};

// Child constructor
function Dog(name, breed) {
    Animal.call(this, name);  // Call parent constructor
    this.breed = breed;
}

// Set up inheritance
Dog.prototype = Object.create(Animal.prototype);
Dog.prototype.constructor = Dog;

// Add child-specific method
Dog.prototype.bark = function() {
    return `${this.name} says Woof!`;
};

const rex = new Dog("Rex", "German Shepherd");
console.log(rex.eat());   // "Rex is eating" (inherited)
console.log(rex.bark());  // "Rex says Woof!" (own method)

## 7. ES6 Classes (Syntactic Sugar)

ES6 класовете са просто синтактична захар върху прототипите - те работят по същия начин "под капака"!

In [ ]:
class Animal {
    constructor(name) {
        this.name = name;
    }
    
    eat() {
        return `${this.name} is eating`;
    }
}

class Dog extends Animal {
    constructor(name, breed) {
        super(name);  // Call parent constructor
        this.breed = breed;
    }
    
    bark() {
        return `${this.name} says Woof!`;
    }
}

const rex = new Dog("Rex", "German Shepherd");
console.log(rex.eat());   // "Rex is eating"
console.log(rex.bark());  // "Rex says Woof!"

// Still uses prototypes under the hood
console.log(rex.__proto__ === Dog.prototype);  // true

## 8. Visual Prototype Chain Example

```
r+--------------------+
|   rex (instance)   |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|   Dog.prototype    |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|  Animal.prototype  |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|  Object.prototype  |
|--------------------|
|     __proto__      |
+--------------------+
          |
          v
+--------------------+
|        null        |
+--------------------+
```

In [ ]:
// Verify the chain
console.log(Object.getPrototypeOf(rex) === Dog.prototype);  // true
console.log(Object.getPrototypeOf(Dog.prototype) === Animal.prototype);  // true
console.log(Object.getPrototypeOf(Animal.prototype) === Object.prototype);  // true
console.log(Object.getPrototypeOf(Object.prototype) === null);  // true

## Summary

- **Прототипите** позволяват наследяване на обекти в JavaScript
- **Прототипната верига** е начинът, по който JavaScript търси свойства
- **Конструкторните функции** използват `prototype`, за да споделят методи
- **`__proto__`** е действителната връзка, **`prototype`** е планът
- **ES6 класовете** са синтактична захар върху прототипите
- Разбирането на прототипите е ключово за овладяването на JavaScript!